In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sqlite3
import pandas as pd

def read_sqlite(conn_path, table_name):

    # Define your SQL query to retrieve data from a specific table
    query = "SELECT * FROM %s" %(table_name)
    with sqlite3.connect(conn_path) as conn:
        df = pd.read_sql_query(query, conn)

    return df

conn_path = '/content/drive/MyDrive/CUNY/Capstone/Data/2_wsj_titles/done/article_content/capstone.db'
table_name = 'e_mini_sp500_futuresHistorical_data_with_obtained_indexes_20100104_20231006'
df = read_sqlite(conn_path, table_name)
df = df.drop(['index', 'PMI_Release Date', 'PMI_Time'], axis=1)
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')




print(df.head())

              Close     Open     High      Low       Vol.  Change %  SOFR  \
Date                                                                        
2010-01-04  1128.75  1113.75  1129.75  1113.25  1291254.0  0.000000   NaN   
2010-01-05  1132.25  1128.50  1133.00  1125.00  1378593.0  0.310078   NaN   
2010-01-06  1133.00  1132.00  1135.50  1127.25  1259921.0  0.066240   NaN   
2010-01-07  1137.50  1133.00  1138.75  1127.00  1567025.0  0.397176   NaN   
2010-01-08  1141.50  1137.25  1141.75  1131.00  1527666.0  0.351648   NaN   

            HQMCB10YR   WM1NS   WM2NS  ...  Fib_0.618_20  Fib_0.236_100  \
Date                                   ...                                
2010-01-04       5.48  1789.5  8534.2  ...           NaN            NaN   
2010-01-05       5.48  1789.5  8534.2  ...           NaN            NaN   
2010-01-06       5.48  1789.5  8534.2  ...           NaN            NaN   
2010-01-07       5.48  1789.5  8534.2  ...           NaN            NaN   
2010-01-08

In [5]:
# df = pd.read_csv('/content/drive/MyDrive/CUNY/Capstone/Data/market_price/df_index_without_TA.csv')

In [3]:
import pandas as pd

# Sample DataFrame
# Assuming df is your DataFrame
# df = ...

# 1. Remove rows before 1/3/2012
df = df[df.index >= '2011-08-01']


# 3. Forward fill null values in specified columns
columns_to_ffill = ['Vol.', 'SOFR', 'DCOILWTICO', 'DTWEXBGS', 'DEXUSEU', 'DEXJPUS', 'DEXCHUS', 'DEXUSUK', 'PMI_Actual', 'PMI_Forecast', 'PMI_Previous', 'far_month_openInterest_0', 'near_month_openInterest', 'total_month_openInterest', 'VIXCLS', 'OBV']
df[columns_to_ffill] = df[columns_to_ffill].fillna(method='ffill')


# 5. Check for any remaining null values
null_values = df.isnull().sum()
print(null_values.sum())


0


<ipython-input-3-c6608d045995>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns_to_ffill] = df[columns_to_ffill].fillna(method='ffill')


In [4]:
# Description of the DataFrame
description = df.describe()

# Summary statistics for each column
summary = df.describe(include='all')

# Display the description and summary
print("Description:")
print(description)
print("\nSummary:")
print(summary)
print(df.info())


Description:
             Close         Open         High          Low          Vol.  \
count  3209.000000  3209.000000  3209.000000  3209.000000  3.209000e+03   
mean   2688.714899  2687.424780  2705.672907  2668.394210  1.585297e+06   
std    1002.959838  1003.150775  1010.436840   994.935584  6.483168e+05   
min    1086.250000  1087.500000  1119.750000  1068.000000  0.000000e+00   
25%    1938.500000  1938.500000  1951.750000  1923.250000  1.190834e+06   
50%    2487.500000  2482.250000  2504.500000  2466.250000  1.514278e+06   
75%    3451.750000  3448.000000  3483.500000  3424.500000  1.885278e+06   
max    4786.000000  4785.250000  4808.250000  4770.500000  6.285917e+06   

          Change %         SOFR    HQMCB10YR         WM1NS         WM2NS  ...  \
count  3209.000000  3209.000000  3209.000000   3209.000000   3209.000000  ...   
mean      0.043925     1.028948     3.667485   7716.855220  14777.434621  ...   
std       1.095728     1.338036     0.812824   7272.256672   3992.70

In [5]:
# List of columns with data type 'object'
object_columns = df.select_dtypes(include=['object']).columns.tolist()

print(object_columns)


[]


In [6]:
import pandas as pd

delta_df = df.copy()  # Create a copy of the DataFrame

# Calculate daily changes for all numerical columns
for col in delta_df.select_dtypes(include=['number']):
    delta_df[f"{col}_delta"] = delta_df[col].diff()  # Calculate deltas

# Handle the first row (set first deltas to 0)
delta_df.iloc[0, delta_df.columns.str.endswith('_delta')] = 0

# Columns to exclude from forward fill
exclude_columns = ['SOFR_delta', 'far_month_openInterest_0_delta', 'near_month_openInterest_delta']

# Replace 0 values with forward-filled values in delta columns, excluding specified columns
delta_df.loc[:, delta_df.columns.str.endswith('_delta') & ~delta_df.columns.isin(exclude_columns)] = delta_df.loc[:, delta_df.columns.str.endswith('_delta') & ~delta_df.columns.isin(exclude_columns)].replace(to_replace=0, method='ffill')


<ipython-input-6-4969476f7573>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  delta_df[f"{col}_delta"] = delta_df[col].diff()  # Calculate deltas
<ipython-input-6-4969476f7573>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  delta_df[f"{col}_delta"] = delta_df[col].diff()  # Calculate deltas


In [10]:
import pandas as pd

# Assuming delta_df is defined and contains the DataFrame you want to save

delta_df.to_csv('/content/drive/MyDrive/CUNY/Capstone/Data/market_price/e_mini_sp500_futuresHistorical_data_with_obtained_indexes_cleand_20110801_20231006.csv')


In [ ]:
# import sqlite3
# from datetime import datetime, timezone
# import pandas as pd
# import time

# def save_to_sqlite(df, conn_path, table_name):
#     # Use a context manager to handle the connection
#     with sqlite3.connect(conn_path) as conn:
#         df.to_sql(table_name, conn, if_exists='replace')

#     save_time = time.time()
#     save_time_datetime = datetime.fromtimestamp(save_time, tz=timezone.utc)
#     save_time_str = save_time_datetime.strftime('%Y-%m-%d %H:%M:%S')

#     print("DataFrame has been saved on %s" % save_time_str)

# conn_path = '/content/drive/MyDrive/CUNY/Capstone/Data/2_wsj_titles/done/article_content/capstone.db'
# table_name = 'e_mini_sp500_futuresHistorical_data_with_obtained_indexes_cleand_20110801_20231006'

# save_to_sqlite(df, conn_path, table_name)


In [7]:
import pandas as pd
delta_df.to_csv("delta_df.csv")